# 18 DYNAMIC PROGRAMMING

Dynamic programming was invented by Richard Bellman in the early 1950s.

Dynamic programming is a method for efficiently solving problems that exhibit the characteristics of <b>overlapping subproblems and optimal substructure</b>.

A problem has <b>optimal substructure</b> if <b>a globally optimal</b> solution can be found by <b>combining</b> optimal solutions to <b>local subproblems</b>.

A problem has <b>overlapping subproblems<b> if an optimal solution involves solving
<b>the same problem</b> multiple times.

## 18.1 Fibonacci Sequences, Revisited

In Chapter 4, we looked at a straightforward recursive implementation of the Fibonacci function, shown here in Figure 18.1.

In [ ]:
#Page 252, Figure 18.1
def fib(n):
    """Assumes n is an int >= 0
       Returns Fibonacci of n"""
    if n == 0 or n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

Look at the tree of calls  associated with the invocation fib(6).
<img src="./img/18.2.PNG"/>
Notice that we are computing the same values over and over again。

It doesn’t require a genius to think that it might be a good idea to record the value returned by the first call, and then look it up rather than compute it each time it is needed. 

This is called <b>memoization</b>, and is the key idea behind dynamic programming.

Figure 18.3 contains an implementation of Fibonacci based on this idea. The function fastFib has a parameter, ```memo```, that it uses to keep track of the numbers it has already evaluated.

In [1]:
#Page 254, Figure 18.3
def fastFib(n, memo = {}):
    """Assumes n is an int >= 0, memo used only by recursive calls
       Returns Fibonacci of n"""
    if n == 0 or n == 1:
        return 1
    try:
        return memo[n]
    except KeyError:
        result = fastFib(n-1, memo) + fastFib(n-2, memo)
        memo[n] = result
        return result

In [3]:
fastFib(120)

8670007398507948658051921

the time complexity of fastFib(n) is O(n)

## 18.2 Dynamic Programming and the 0/1 Knapsack Problem



Dynamic programming provides a practical method for solving most 0/1 knapsack problems in a reasonable amount of time.

The key idea is to think about exploring the space of possible solutions by constructing <b>a rooted binary tree that enumerates all states that satisfy the weight constraint</b>.

A rooted binary tree is an acyclic directed graph in which
<ul>
<li>There is exactly one node with no parents. This is called the <b>root</b>.
<li>Each non-root node has <b>exactly one parent</b>.
<li>Each node has  <b>at most two children</b>. A childless node is called a <b>leaf</b>.
</ul>

Each node in the search tree for the 0/1 knapsack problem is labeled with a quadruple that denotes a partial solution to the knapsack problem.

The elements of the quadruple are:
<ul>
<li>A set of items to be taken,
<li>The list of items for which a decision has not been made,
<li>The total value of the items in the set of items to be taken (this is merely an optimization, since the value could be computed from the set), and
<li>The remaining space in the knapsack. (Again, this is an optimization since it is merely the difference between the weight allowed and the weight of all the items taken so far.)
</ul>

The tree is built <b>top-down</b> starting with the root.

One element is selected from the still-to-be-considered items. If <b>there is room for that item</b> in the knapsack, <b>a node is constructed</b> that reflects the consequence of choosing to take that item. By convention, we draw that node as <b>the left child</b>. 

<b>The right child</b> shows the consequences of choosing not to take that item. 

The process is then applied recursively until either the knapsack is full or there are no more items to consider. 

Because <b>each edge</b> represents<b> a decision</b> (to take or not to take an item), such trees are called <b>decision trees</b>

Figure 18.4 is a table describing a set of items
<img src="./img/18.4.PNG"/>
Figure 18.5 is a decision tree for deciding which of those items to take under the assumption that the knapsack has a maximum weight of 5.
<img src="./img/18.5.PNG"/>
In Figure 18.5, <b>the numbers that precede the colon</b> in each node indicate one order in which the nodes could be generated. This particular ordering is called <b>left-first depth-first</b>.

At each node we attempt to generate a left node. If that is impossible, we attempt to generate a right node.
<img src="./img/18.5.1.PNG"/>
<img src="./img/18.5.2.PNG"/>
and any leaf node with the greatest value represents an optimal solution
<img src="./img/18.5.3.PNG"/>


Figure 18.6 contains such an implementation. It uses class Item from Figure 17.2.

The function <b>maxVal</b>:
<ul>
<li>returns two values: the set of items chosen and the total value of those items。
<li>called with two arguments, corresponding to the second and fourth elements of the labels of the nodes in the tree:
<ul>
<li><b>toConsider</b>.:Those items that nodes higher up in the tree (corresponding to earlier calls in the recursive call stack) have not yet considered.
<li><b>avail</b>: The amount of space still available.
</ul>
</ul>
it uses the local variable ```result``` to record the best solution found so far.

In [16]:
class Item(object):
    def __init__(self, n, v, w):
        self.name = n
        self.value = float(v)
        self.weight = float(w)
    def getName(self):
        return self.name
    def getValue(self):
        return self.value
    def getWeight(self):
        return self.weight
    def __str__(self):
        result = '<' + self.name + ', ' + str(self.value) + ', ' + str(self.weight) + '>'
        return result

#Page 257, Figure 18.6
def maxVal(toConsider, avail):
    """Assumes toConsider a list of items, avail a weight
       Returns a tuple of the total weight of a solution to the
         0/1 knapsack problem and the items of that solution"""
    if toConsider == [] or avail == 0:
        result = (0, ())  # the total value of those items=0， the set of items chosenwwwwwwwwwwwwwwwwwww/
    elif toConsider[0].getWeight() > avail:
        #Explore right branch only
        result = maxVal(toConsider[1:], avail)
    else:
        nextItem = toConsider[0]
        
        #Explore left branch
        withVal, withToTake = maxVal(toConsider[1:],avail - nextItem.getWeight())
        withVal += nextItem.getValue()
        
        #Explore right branch
        withoutVal, withoutToTake = maxVal(toConsider[1:], avail)
        
        #Choose better branch
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    return result

def smallTest():
    names = ['a', 'b', 'c', 'd']
    vals = [6, 7, 8, 9]
    weights = [3, 3, 2, 5]
    Items = []
    for i in range(len(vals)):
        Items.append(Item(names[i], vals[i], weights[i]))
   
    val, taken = maxVal(Items, 5)
    
    for item in taken:
        print(item)
    
    print('Total value of items taken =', val)            

In [13]:
smallTest()

<c, 8.0, 2.0>
<b, 7.0, 3.0>
Total value of items taken = 15.0


The code in Figure 18.7 makes it convenient to test maxVal. It randomly generates a list of Items of a specified size.

In [19]:
import   random
#Page 258, Figure 18.7
def buildManyItems(numItems, maxVal, maxWeight):
    items = []
    for i in range(numItems):
        items.append(Item(str(i),
                          random.randint(1, maxVal),
                          random.randint(1, maxWeight)))
    return items

def bigTest(numItems):
    items = buildManyItems(numItems, 10, 10)
    val, taken = maxVal(items, 40)
    print('Items Taken')
    for item in taken:
        print( item)
    print( 'Total value of items taken =', val)

In [20]:
bigTest(10)

Items Taken
<9, 4.0, 4.0>
<8, 8.0, 6.0>
<7, 2.0, 3.0>
<6, 3.0, 2.0>
<4, 6.0, 6.0>
<3, 3.0, 8.0>
<2, 9.0, 7.0>
<0, 8.0, 4.0>
Total value of items taken = 43.0


#### Dynamic programming solution to knapsack problem

Let’s start by asking whether this program has anything in common with our first implementation of Fibonacci. In particular, is there optimal substructure and are there overlapping subproblems?

<b>Optimal substructure</b> is visible both in Figure 18.5 and in Figure 18.6. 

Each parent node combines the solutions reached by its children to derive an optimal solution for the subtree rooted at that parent. 

This is reflected in Figure 18.6 by the code following the comment 
```python
#Choose better branch
if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
```

Are there also <b>overlapping subproblems?</b> At first glance, the answer seems to be “no.” At each level of the tree we have a different set of available items to consider. This implies that if common subproblems do exist, they must be at the same level of the tree. And indeed at each level of the tree each node has the same set of items to consider taking. However, we can see by looking at the labels in Figure 18.5 that each node at a level represents a different set of choices about the items considered higher in the tree.

Think about <b>what problem is being solved at each node</b>. 

The problem being solved is finding the optimal items to take from those left to consider, <b>given the remaining available weight</b>. 

The <b>available weight</b> depends upon <b>the total weight</b> of the items taken, but not on which items are taken or the total value of the items taken.

So, for example, in Figure 18.5, nodes 2 and 7 are actually solving the same problem: deciding which elements of [c,d] should be taken, given that the available weight is 2.
<img src="./img/18.8.0.PNG"/>

The code in Figure 18.8 exploits <b>the  optimal substructure and overlapping subproblems</b> to provide <b>a dynamic programming solution</b> to the 0/1 knapsack problem.

An extra parameter, <b>memo</b>, has been added to keep track of <b>solutions to subproblems that have already been solved</b>.

It is implemented using <b>a dictionary</> with <b>a key</> constructed from <b>the length of toConsider and the
available weight</>.
```python
memo[(len(toConsider), avail)] = result
```





In [ ]:

#Page 259, Figure 18.8
def fastMaxVal(toConsider, avail, memo = {}):
    """Assumes toConsider a list of items, avail a weight
         memo used only by recursive calls
       Returns a tuple of the total weight of a solution to the
         0/1 knapsack problem and the items of that solution"""
   
   if (len(toConsider), avail) in memo: #  overlapping
        result = memo[(len(toConsider), avail)]
   
    elif toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getWeight() > avail:
        #Explore right branch only
        result = fastMaxVal(toConsider[1:], avail, memo)
    else:
        nextItem = toConsider[0]
        
        #Explore left branch  
        withVal, withToTake =fastMaxVal(toConsider[1:],
                            avail - nextItem.getWeight(), memo)
        withVal += nextItem.getValue()
        
        #Explore right branch
        withoutVal, withoutToTake = fastMaxVal(toConsider[1:],
                                                avail, memo)
        #Choose better branch
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    
    memo[(len(toConsider), avail)] = result  # 
    return result

Figure 18.9 shows the number of calls made when we ran the code on problems of various sizes.
<img src="./img/18.9.PNG"/>


The running time of fastMaxVal is governed by the number of distinct ```<Consider, avail>``` pairs generated

The number of possible values of ```toConsider``` is bounded by ```len(items)```.

The number of possible values of ```avail``` is more difficult to characterize. It is bounded from above by the maximum number of distinct totals of weights of the items that the knapsack can hold.


The growth is hard to quantify, but it is clearly far less than exponential.

This algorithm falls into a complexity class called <b>pseudo polynomial</b>

## 18.3 Dynamic Programming and Divide-and-Conquer

<b>Divide-and-conquer</b> algorithms are based upon finding subproblems that are <b>substantially smaller</b> than the original problem.

<b>dynamic programming</b> involves solving problems that are only <b>slightly smaller</b> than the
original problem.

Another important distinction is that the efficiency of divide-and-conquer algorithms <b>does not depend upon structuring</b> the algorithm so that the same problems are solved repeatedly.

dynamic programming is efficient only when the number of distinct <b>subproblems is significantly smaller than</b> the
total number of subproblems. 



